# Логистическая регрессия с методом максимального правдоподобия
Этот ноутбук предназначен для запуска в Google Colab. Он включает все этапы предобработки данных, обучения модели и оценки параметров с помощью метода максимального правдоподобия.

## 1. Импорт необходимых библиотек

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
%matplotlib inline

## 2. Загрузка и предварительный анализ данных

In [ ]:
# Загрузка данных (используем публичный датасет churn) 
url = 'https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/titanic/train.csv'
df = pd.read_csv(url)

# Просмотр первых 5 строк
df.head()

## 3. Очистка данных и предобработка

In [ ]:
# Преобразуем названия колонок в нижний регистр и заменяем пробелы
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Заполняем пропущенные значения медианой
df['age'].fillna(df['age'].median(), inplace=True)

# Преобразуем категориальные переменные в числовые
df['sex'] = (df['sex'] == 'male').astype(int)
df['embarked'] = df['embarked'].fillna('S')
df = pd.get_dummies(df, columns=['embarked'], drop_first=True)

# Выводим типы данных
df.dtypes

## 4. Разделение данных на обучающую и тестовую выборки

In [ ]:
# Выбираем признаки и целевую переменную
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked_Q', 'embarked_S']
X = df[features]
y = df['survived']

# Разделяем на обучающий и тестовый набор
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Проверяем размерность
X_train.shape, X_test.shape

## 5. Реализация метода максимального правдоподобия

In [ ]:
def log_likelihood(theta, X, y):
    z = np.dot(X, theta)
    log_l = np.sum(y * z - np.log(1 + np.exp(z)))
    return -log_l  # Минимизируем отрицательное правдоподобие

## 6. Обучение модели логистической регрессии

In [ ]:
def train_mle(X, y):
    n_features = X.shape[1]
    initial_theta = np.zeros(n_features)
    result = minimize(log_likelihood, initial_theta, args=(X, y), method='BFGS')
    return result.x

# Добавляем единичный столбец для свободного члена
X_train_mle = np.hstack((np.ones((X_train.shape[0], 1)), X_train))

# Обучаем модель
theta_opt = train_mle(X_train_mle, y_train)
theta_opt

## 7. Интерпретация коэффициентов

In [ ]:
feature_names = ['Intercept'] + features
for name, coef in zip(feature_names, theta_opt):
    print(f'{name}: {coef:.4f}')